In [1]:
# Import dependencies
import pandas as pd
import numpy as np

In [2]:
# Read original database
hw_df = pd.read_csv('US_counties_COVID19_health_weather_data_05292020.csv')

In [3]:
# Check on datatypes in hw_df
hw_df_dt = pd.DataFrame(hw_df.dtypes).reset_index()
hw_df_dt.to_csv('hw_df_dt.csv')

In [4]:
# Find the most recent date of data collection
hw_df['date'] = pd.to_datetime(hw_df['date'])
max_date = hw_df['date'].max()
max_date

Timestamp('2020-05-29 00:00:00')

In [5]:
# Make sure all the counties have info for most recent date of data collection
max_date == hw_df[['fips','date']].groupby('fips').max().reset_index()['date'].unique()[0]

True

In [6]:
# Filter the original dataframe to the max_date
hw_date = hw_df[hw_df['date'] == max_date].reset_index(drop=True)
len(hw_date)

2957

In [7]:
# Find US average for each parameter - if applicable
col_w_US_mean = pd.DataFrame(hw_date.mean()).dropna().index.tolist()
col_wo_US_mean = set(hw_date.columns) - set(col_w_US_mean)
hw_US_mean_df = pd.DataFrame(hw_date[col_w_US_mean].mean()).T
hw_US_mean_df

,cases,deaths,lat,lon,total_population,area_sqmi,population_density_per_sqmi,num_deaths,years_of_potential_life_lost_rate,percent_fair_or_poor_health,...,wind_speed,max_wind_speed,wind_gust,precipitation,fog,rain,snow,hail,thunder,tornado
0,588.700372,34.620223,38.232154,-91.526341,107861.591478,1080.321265,235.121584,1364.806163,8557.013059,18.027857,...,4.632087,9.15411,20.390778,0.096304,0.037486,0.219992,0.0,0.0,0.035214,0.0


In [8]:
# Check on datatypes on hw_US_mean_df
hw_US_mean_df_dt = pd.DataFrame(hw_US_mean_df.dtypes).reset_index()
hw_US_mean_df_dt

,index,0
0,cases,float64
1,deaths,float64
2,lat,float64
3,lon,float64
4,total_population,float64
...,...,...
195,rain,float64
196,snow,float64
197,hail,float64
198,thunder,float64


In [9]:
# Find the columns where datatype changed after averaging in US
US_dt_merge = hw_df_dt.merge(hw_US_mean_df_dt, on='index', how='left')
US_dt_contradict = US_dt_merge[US_dt_merge['0_x'] != US_dt_merge['0_y']].dropna(subset=['0_y']).reset_index(drop=True)
US_dt_contradict.columns=['col_name', 'orig', 'converted']
US_dt_contradict

,col_name,orig,converted
0,cases,int64,float64
1,deaths,int64,float64
2,total_population,int64,float64
3,num_uninsured,int64,float64
4,num_some_college,int64,float64
5,population,int64,float64
6,num_unemployed_CHR,int64,float64
7,labor_force,int64,float64
8,num_single_parent_households_CHR,int64,float64
9,num_households_CHR,int64,float64


In [10]:
# Convert hw_US_mean_df to the original data types
dt_orig = pd.Series(US_dt_contradict.orig.values,index=US_dt_contradict.col_name).to_dict()
hw_US_mean_df = hw_US_mean_df.astype(dt_orig)
hw_US_mean_df

,cases,deaths,lat,lon,total_population,area_sqmi,population_density_per_sqmi,num_deaths,years_of_potential_life_lost_rate,percent_fair_or_poor_health,...,wind_speed,max_wind_speed,wind_gust,precipitation,fog,rain,snow,hail,thunder,tornado
0,588,34,38.232154,-91.526341,107861,1080.321265,235.121584,1364.806163,8557.013059,18.027857,...,4.632087,9.15411,20.390778,0.096304,0.037486,0.219992,0.0,0.0,0.035214,0.0


In [11]:
# Group data by state, and get the average value
hw_state = hw_date.groupby(['state']).mean().reset_index()

In [12]:
# Check on datatypes on hw_state
hw_state_dt = pd.DataFrame(hw_state.dtypes).reset_index()
hw_state_dt

,index,0
0,state,object
1,cases,float64
2,deaths,float64
3,lat,float64
4,lon,float64
...,...,...
211,min_temp_15d_avg,float64
212,dewpoint_3d_avg,float64
213,dewpoint_5d_avg,float64
214,dewpoint_10d_avg,float64


In [13]:
# Find the columns where datatype changed after averaging in states
state_dt_merge = hw_df_dt.merge(hw_state_dt, on='index', how='left')
state_dt_contradict = state_dt_merge[state_dt_merge['0_x'] != state_dt_merge['0_y']].dropna(subset=['0_y']).reset_index(drop=True)
state_dt_contradict.columns=['col_name', 'orig', 'converted']
state_dt_contradict

,col_name,orig,converted
0,cases,int64,float64
1,deaths,int64,float64
2,total_population,int64,float64
3,num_uninsured,int64,float64
4,num_some_college,int64,float64
5,population,int64,float64
6,num_unemployed_CHR,int64,float64
7,labor_force,int64,float64
8,num_single_parent_households_CHR,int64,float64
9,num_households_CHR,int64,float64


In [14]:
# Convert hw_state to the original data types
dt_orig = pd.Series(state_dt_contradict.orig.values,index=state_dt_contradict.col_name).to_dict()
hw_state = hw_state.astype(dt_orig)
hw_state

,state,cases,deaths,lat,lon,total_population,area_sqmi,population_density_per_sqmi,num_deaths,years_of_potential_life_lost_rate,...,max_temp_10d_avg,max_temp_15d_avg,min_temp_3d_avg,min_temp_5d_avg,min_temp_10d_avg,min_temp_15d_avg,dewpoint_3d_avg,dewpoint_5d_avg,dewpoint_10d_avg,dewpoint_15d_avg
0,Alabama,254,9,32.883840,-86.709923,72256,755.916771,91.310922,1220.761194,10931.071313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alaska,25,0,61.355358,-146.473195,41276,27494.107559,12.989270,493.312500,9881.006189,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arizona,1231,59,33.676042,-111.462832,448571,7572.710164,54.588037,5359.600000,9409.741075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arkansas,88,1,34.930317,-92.432799,40044,694.689033,55.841659,647.229730,10595.502891,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,California,1877,72,37.781489,-120.727785,677985,2664.511210,706.573596,6321.327273,6470.845704,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Colorado,426,23,38.902289,-105.519107,89206,1644.072717,169.372934,1086.958333,6961.519589,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Connecticut,5192,483,41.614420,-72.657515,448571,605.342219,725.316122,4395.375000,5841.181651,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Delaware,3061,118,39.107831,-75.537142,311565,649.577328,604.300758,4224.666667,7912.815499,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,District of Columbia,8538,460,38.904742,-77.016296,659009,61.145040,10777.799777,8387.000000,8111.250945,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Florida,812,36,28.946244,-82.698476,297529,800.580764,356.472644,3932.179104,8704.984293,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Fill the null values in state averages with national averages
columns = hw_US_mean_df.columns.to_list()
columns.remove('presence_of_water_violation')
for col in columns:
    hw_state[col].fillna(hw_US_mean_df[col][0], inplace=True)

In [16]:
# Fill the null values in date filtered dataframe with state averages dataframe
hw_merged = hw_date.merge(hw_state,on='state', how='left')
x_col = [col for col in hw_merged.columns if col.endswith('_x')==True]
y_col = [col for col in hw_merged.columns if col.endswith('_y')==True]

for col in x_col:
    hw_merged[col] = np.where(hw_merged[col].isnull()==False, hw_merged[col], hw_merged[col.replace("_x","_y")])
    
hw_merged.drop(columns=y_col, inplace=True)
hw_merged.columns = [col.replace("_x","") for col in hw_merged.columns.to_list()]

In [17]:
# QC on the process to make sure all the null values has been replaced except for the columns with all values as NA values
hw_merged_na = []
for col in hw_merged.columns:
    if hw_merged[col].isnull().nunique() == 1 and hw_merged[col].isnull().unique()[0] == False:
        continue
    else:
        hw_merged_na.append(col)
        
set(hw_merged_na)-set(col_wo_US_mean)
# 'presence_of_water_violation' is a boolean field - so out of the game!
hw_merged_na.remove('presence_of_water_violation')

In [18]:
hw_merged_na

['CALL',
 'station_name',
 'precip_flag',
 'mean_temp_3d_avg',
 'mean_temp_5d_avg',
 'mean_temp_10d_avg',
 'mean_temp_15d_avg',
 'max_temp_3d_avg',
 'max_temp_5d_avg',
 'max_temp_10d_avg',
 'max_temp_15d_avg',
 'min_temp_3d_avg',
 'min_temp_5d_avg',
 'min_temp_10d_avg',
 'min_temp_15d_avg',
 'dewpoint_3d_avg',
 'dewpoint_5d_avg',
 'dewpoint_10d_avg',
 'dewpoint_15d_avg',
 'date_stay_at_home_announced',
 'date_stay_at_home_effective']

In [19]:
set(col_wo_US_mean)-set(hw_merged_na)

{'county',
 'date',
 'fips',
 'state',
 'stay_at_home_announced',
 'stay_at_home_effective'}

In [20]:
# Drop the columns that are all NA values
#hw_merged.drop(columns=hw_merged_na, inplace=True)

In [21]:
# Replace the columns that are all NA values with "0"
hw_merged.fillna('0', inplace=True)
hw_merged.index.name='index'

In [22]:
hw_merged.drop(['lat', 'lon'], axis=1, inplace=True)

In [23]:
# Remove weather data
weather_df =['km_to_closest_station', 'CALL', 'ELEV_M', 'station_id','station_name','mean_temp', 'min_temp',\
             'max_temp', 'dewpoint', 'sea_level_pressure', 'station_pressure', 'visibility', \
            'wind_speed', 'max_wind_speed', 'wind_gust', 'precipitation', 'precip_flag', 'fog','rain', 'snow', 'hail', 'thunder',\
            'tornado', 'mean_temp_3d_avg', 'mean_temp_5d_avg', 'mean_temp_10d_avg', 'mean_temp_15d_avg', 'max_temp_3d_avg', 'max_temp_5d_avg',\
            'max_temp_10d_avg', 'max_temp_15d_avg', 'min_temp_3d_avg', 'min_temp_5d_avg', 'min_temp_10d_avg', 'min_temp_15d_avg', \
            'dewpoint_3d_avg', 'dewpoint_5d_avg', 'dewpoint_10d_avg', 'dewpoint_15d_avg', 'date_stay_at_home_announced', \
            'date_stay_at_home_effective']

In [24]:
hw_merged.drop(weather_df, axis=1, inplace=True)

In [25]:
int(hw_merged['fips'][0])

1001

In [26]:
for i in hw_merged['fips']:
    try:
        int(i)
    except:
        print(i)

KSC
NYC


In [27]:
#hw_merged.to_csv('US_counties_COVID19_health_weather_data_replaced_na_05292020.csv')